# Virl Client Demo 2

## Setup

In [55]:
import urllib3
from virl2_client import ClientLibrary
import env
import logging
import sys
import os
import netmiko

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(message)s')
log = logging.getLogger(__name__)

# Read environment variables
cml_server_url = env.config['CML_SERVER_URL']
cml_server_ip = env.config['CML_SERVER_IP']
cml_username = env.config['CML_USERNAME']
cml_password = env.config['CML_PASSWORD']
LAB_TITLE = "CML_IS_AWESOME"

### Client

In [56]:
client = ClientLibrary(cml_server_url, cml_username, cml_password, ssl_verify=False, raise_for_auth_failure=True, allow_http=True)
log.info("LOGGING INFO: Successfully connected with CML through the API")

SSL Verification disabled
LOGGING INFO: Successfully connected with CML through the API


## Read in the config file

In [57]:
config_files = os.listdir(path='./config')
config_files = [file for file in config_files if ".txt" in file]
log.info("LOGGING INFO: Successfully read in the config files")

routers = []
for file in config_files:
    routers.append(file[:-4])

LOGGING INFO: Successfully read in the config files


## Create a new lab in CML

In [58]:
lab = client.create_lab(title=LAB_TITLE)
log.info("LOGGING INFO: Successfully created the lab in CML")

LOGGING INFO: Successfully created the lab in CML


## Create the nodes in the lab

In [59]:
coordinates = [(0,0), (200, 0), (200,200), (0, 200)]
coordinates_counter = 0 
for router in routers:
    x, y = coordinates[coordinates_counter]
    lab.create_node(label=router, node_definition='csr1000v', populate_interfaces=8, x=x, y=y)
    coordinates_counter += 1
log.info("LOGGING INFO: Successfully created the nodes in the lab")

Waiting for lab 0d624a to converge
Lab 0d624a has booted
Waiting for lab 0d624a to converge
Lab 0d624a has booted
Waiting for lab 0d624a to converge
Lab 0d624a has booted
Waiting for lab 0d624a to converge
Lab 0d624a has booted
LOGGING INFO: Successfully created the nodes in the lab


## Configure the nodes in the lab

In [60]:
for node in lab.nodes():
    config = open(f"./config/{node.label}.txt", 'r').read()
    node.config = config
log.info("LOGGING INFO: Successfully configured the nodes in the lab")

Added interface Interface: GigabitEthernet1
Added interface Interface: GigabitEthernet4
Added interface Interface: GigabitEthernet1
Added interface Interface: GigabitEthernet2
Added interface Interface: GigabitEthernet2
Added interface Interface: GigabitEthernet4
Added interface Interface: Loopback0
Added interface Interface: GigabitEthernet1
Added interface Interface: GigabitEthernet3
Added interface Interface: GigabitEthernet4
Added interface Interface: Loopback0
Added interface Interface: Loopback0
Added interface Interface: GigabitEthernet3
Added interface Interface: Loopback0
Added interface Interface: GigabitEthernet2
Added interface Interface: GigabitEthernet4
Added interface Interface: GigabitEthernet3
Added interface Interface: GigabitEthernet2
Added interface Interface: GigabitEthernet3
Added interface Interface: GigabitEthernet1
LOGGING INFO: Successfully configured the nodes in the lab


## Connect the nodes to each other

In [61]:
interface_pairs = [('001', '002'), ('002', '003'), ('003', '004'), ('004', '001')]
for intf1, intf2 in interface_pairs:
    for interface in lab.interfaces():
        if intf1 in interface.node.label and '2' in interface.label:
            interface1 = interface
        if intf2 in interface.node.label and '3' in interface.label:
            interface2 = interface
    lab.create_link(interface1, interface2)
log.info("LOGGING INFO: Successfully created links between the nodes")

Waiting for lab 0d624a to converge
Lab 0d624a has booted
Waiting for lab 0d624a to converge
Lab 0d624a has booted
Waiting for lab 0d624a to converge
Lab 0d624a has booted
Waiting for lab 0d624a to converge
Lab 0d624a has booted
LOGGING INFO: Successfully created links between the nodes


## Get lab testbed

In [62]:
pyats_testbed = lab.get_pyats_testbed()
# Write the YAML testbed out to a file
with open("lab_testbed.yaml", "w") as f: 
    f.write(pyats_testbed)
log.info("LOGGING INFO: Successfully obtained a testbed file")

LOGGING INFO: Successfully obtained a testbed file


## Edit Testbed

In [ ]:
#Add credentials to yaml file
doc = yaml.load(pyats_testbed)
#doc["devices"]["terminal_server"]["connections"]["cli"]["username"] = cml_username
#doc["devices"]["terminal_server"]["connections"]["cli"]["password"] = cml_password
#doc["devices"]["terminal_server"]["credentials"]["default"]["username"] = cml_username
#doc["devices"]["terminal_server"]["credentials"]["default"]["password"] = cml_password

# Write the YAML testbed out to a file
with open("lab_testbed.yaml", "w") as f: 
    yaml.dump(doc,f)

## Make them start

In [63]:
lab.start()

Waiting for lab 0d624a to converge
Lab has not converged, attempt 0/500, waiting...
Lab has not converged, attempt 10/500, waiting...
Lab has not converged, attempt 20/500, waiting...
Lab has not converged, attempt 30/500, waiting...
Lab 0d624a has booted


## Run commands with Netmiko

In [ ]:
our_lab = client.find_labs_by_title(LAB_TITLE)[0]
xr_node = our_lab.get_node_by_label('R003')

# open the Netmiko connection via the terminal server
# (SSH to the controller connects to the terminal server)
c = netmiko.ConnectHandler(device_type = 'terminal_server',
                           host = cml_server_ip,
                           username = cml_username,
                           password = cml_password)

# send CR, get a prompt on terminal server
c.write_channel('\r')

# open the connection to the console
c.write_channel(f'open /{our_lab.id}/{xr_node.id}/0\r')

# router login
# this makes an assumption that it's required to login
c.write_channel('\r')
c.write_channel(cml_username + '\r')
c.write_channel(cml_password + '\r')

# get the list of interfaces
result = c.send_command('show ip int brief')
print(result)

In [ ]:
# get the inventory
result = c.send_command('show inventory')
c.find_prompt()
print(result)

In [ ]:
lab.stop()
lab.wipe()
lab.remove()